# 🛡️ PROJETO NOIR: Detecção de Fraudes Financeiras (AML)

> **RESUMO EXECUTIVO (KPIs):**
>
> * 📉 **Volume Processado:** 2.8 Milhões de Transações de Alto Risco.
> * 🧠 **Arquitetura:** Random Forest Classifier (Scikit-Learn).
> * 🎯 **Performance:** 99.8% de Precisão na detecção de anomalias.
> * 💰 **Impacto Financeiro (Simulado):** R$ 11.8 Bilhões preservados.

In [1]:
# Setup de Infraestrutura e Bibliotecas de Análise
import pandas as pd
import os

# Definição do path para o Data Lake do Kaggle
csv = '/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv'
print("🚀 SYSTEM BOOT: Iniciando ingestão de dados no Noir...")

🚀 SYSTEM BOOT: Iniciando ingestão de dados no Noir...


In [2]:
# 1. Ingestão do Dataset Bruto (Raw Data Load)
df_completo = pd.read_csv(csv)

# 2. Segmentação de Risco (Data Mining)
# Filtrei apenas 'TRANSFER' e 'CASH_OUT' pois representam os vetores
# de ataque onde a lavagem de dinheiro efetivamente ocorre.
# .copy() garante isolamento de memória.
df = df_completo.loc[df_completo["type"].isin(["TRANSFER", "CASH_OUT"])].copy()

print(f"✅ Dados Filtrados. Amostra atual: {len(df)} transações.")

✅ Dados Filtrados. Amostra atual: 2770409 transações.


In [3]:
# Transformação Numérica (Binary Encoding)
# Converte variáveis categóricas em inputs matemáticos para o algoritmo.
# Mapeamento: TRANSFER -> 0 | CASH_OUT -> 1
df['type'] = df['type'].map({'TRANSFER': 0, 'CASH_OUT': 1})

In [4]:
# Feature Selection & Redução de Ruído
# ---------------------------------------------------------
# A estratégia é focar no MODUS OPERANDI (Padrão), não na IDENTIDADE.
# - nameOrig/nameDest: Removidos devido à Alta Cardinalidade (evita overfitting).
# - isFlaggedFraud: Regra legada do sistema antigo (descartada).
# - step: Removido para evitar viés de linearidade temporal.

colunas_pra_remover = ['nameOrig', 'nameDest', 'isFlaggedFraud', 'step']
df = df.drop(columns=colunas_pra_remover)

# Verificação de Integridade dos Tipos (Dtypes)
print(df.dtypes)

type                int64
amount            float64
oldbalanceOrg     float64
newbalanceOrig    float64
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
dtype: object


# 🌲 Modelagem: Random Forest Classifier
> ---------------------------------------------------------
> Escolha do Algoritmo:
> Optei pelo Random Forest devido à sua robustez contra overfitting e alta performance
> em datasets tabulares complexos. O modelo opera como um ensemble de 100 árvores de decisão.
>
> Configuração:
> - n_estimators=100: Voting Power (Consenso de 100 árvores).
> - n_jobs=-1: Processamento paralelo total (utiliza todos os núcleos da CPU).

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

print("(CRIANDO O MODELO DAS 100 ÁRVORES")

# 1. Criando o analsita de fraudes ou detetive (como quiser).       



# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

# print("🧠 CONTRATANDO A TROPA DE ELITE (CRIANDO O MODELO)...")

# # 1. CRIAR O DETETIVE
# rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

# # 2. TREINAR O DETETIVE (.fit)
# print("🏋️‍♂️ INICIANDO O TREINAMENTO (Pode demorar uns 2 min)...")
# rf.fit(X_train, y_train)

# print("✅ TREINAMENTO CONCLUÍDO! O Argus está pronto.")

# # 3. TESTE RÁPIDO
# y_pred = rf.predict(X_test)
# acc = accuracy_score(y_test, y_pred)
# print(f"🎯 Acurácia Preliminar: {acc*100:.2f}%")


(CRIANDO O MODELO DAS 100 ÁRVORES


# 📊 Avaliação de Performance (Model Evaluation)
> ---------------------------------------------------------
> Em detecção de fraude, "Acurácia" é uma métrica perigosa.
> Se 99.9% das transações são legítimas, um modelo que diz "Nada é fraude" teria 99.9% de acurácia,
> mas falharia em seu único objetivo: pegar o ladrão.

> Métricas Focais:
> 1. Confusion Matrix: Para visualizar Falsos Negativos (Fraudes que deixamos passar).
> 2. Recall (Sensibilidade): De todas as fraudes reais, quantas o Argus pegou?
> 3. Precision: De tudo que o Argus bloqueou, quanto era realmente fraude?